# Image Classification

Pada notebook ini, kita akan belajar tentang:
* Apa itu *image classification*?
* Tantangan dalam *image classification*
* *Image feature*
* Modeling untuk *image classification*
* Studi kasus: *Fashion-MNIST image classification*

## Apa Itu *Image Classification*?

Klasifikasi gambar atau yang biasa disebut dengan *image classification* adalah sebuah *task* dalam *computer vision* untuk memproses sebuah gambar dan membuat komputer untuk mengenali atau mengklasifikasi konten dalam gambar ke dalam sebuah *class* (dalam bentuk probabilitas).

![](../../../../assets/images/cat-classification.png)

<!-- ```{figure} ../../../../assets/images/cat-classification.png
:name: cat-classification
:width: 50%

Gambar diambil dari [Coursera](https://www.coursera.org/learn/introduction-computer-vision-watson-opencv).
``` -->

*Class* yang dimaksud ini adalah sebuah label atau kategori yang merepresentasikan target variabel yang akan kita prediksi, seperti kucing, mobil, atau lainnya. Beberapa aplikasi dari *image classification* seperti:
* Pengelompokkan foto pada handphone
* Pengelompokkan gambar pada *travel agent*
* Pengenalan wajah
* dan lainnya


## Jenis *Image Classification*

Secara umum, ada 2 jenis *task* klasifikasi:

```{panels}
**Binary Classificastion**
^^^
Klasifikasi yang variabel targetnya terdiri dari 2 jenis. Seperti kucing/anjing, benar/salah, positif/negatif, dan lainnya.
---

**Multiclass Classification**
^^^
Klasifikasi yang variabel targetnya lebih dari 2 jenis. Seperti positif/negatif/netral, angka 1/2/3/dst.
```

![](https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/doc/img/fashion-mnist-sprite.png)
<!-- ```{figure} https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/doc/img/fashion-mnist-sprite.png
:name: fashion-mnist
:width: 50%

Contoh *multiclass classification* pada data [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist).
``` -->

![](../../../../assets/images/cat-dog-binary.png)
<!-- ```{figure} ../../../../assets/images/cat-dog-binary.png
:name: cat-dog-binary
:width: 50%

Contoh *binary classification* diambil dari [Coursera](https://www.coursera.org/learn/introduction-computer-vision-watson-opencv).
``` -->

## Tantangan dalam *Image Classification*

Kita dengan mudah mengenali bahwa objek dalma kedua gambar di bawah ini adalah kucing. Akan tetapi, bisa saja komputer tidak bisa mengenali salah satu gambar tersebut.

![](../../../../assets/images/cats.png)
<!-- ```{figure} ../../../../assets/images/cats.png
:name: cats
:width: 50%

Gambar [1](https://www.npr.org/2019/04/04/709916647/study-your-cat-knows-when-youre-calling-him-he-just-doesnt-care) dan [2](https://static01.nyt.com/images/2021/08/17/science/28cats1/28cats1-mobileMasterAt3x.jpg).
``` -->

Hal ini dikarenakan ada beberapa tantangan yang datang dari ragam gambar/foto. Diantaranya adalah {cite:p}`ibm-image-classification`:
* Illumination
    ![](../../../../assets/images/img-illumination.png)
    <!-- ```{figure} ../../../../assets/images/img-illumination.png
    :name: img-illumination
    ``` -->
* Deformation
    ![](../../../../assets/images/img-deformation.png)
    <!-- ```{figure} ../../../../assets/images/img-deformation.png
    :name: img-deformation
    ``` -->
* Occlusion
    ![](../../../../assets/images/img-occlusion.png)
    <!-- ```{figure} ../../../../assets/images/img-occlusion.png
    :name: img-occlussion
    ``` -->
* Background clutter
    ![](../../../../assets/images/img-bg-clutter.png)
    <!-- ```{figure} ../../../../assets/images/img-bg-clutter.png
    :name: img-bg-clutter
    ``` -->

## *Image Feature*

Untuk komputer bisa memproses sebuah gambar, akan digunakan nilai intensitas sebuah gambar yang direpresentasikan oleh matriks piksel.

![](../../../../assets/images/dog-pixel.png)
<!-- ```{figure} ../../../../assets/images/dog-pixel.png
:name: dog-pixel

Contoh gambar dan reprsentasi matriks piksel ([ref](https://www.coursera.org/learn/introduction-computer-vision-watson-opencv))
``` -->

```{note}
Pada gambar di atas, hanya ada 1 jenis matriks piksel karena gambar dalam format *grayscale*. Pada gambar berwarna, kita akan punyai matriks piksel sesuai dengan jumlah channel warnanya, yaitu 3 jenis, merah, hijau, dan biru (RGB).
```

Jika pada data tabel, umumnya kita memiliki beberapa baris data dengan beberapa kolom yang merupakan fitur dari data. Pada data gambar, fitur digunakan adalah matriks piksel pada gambar tersebut. Hal ini dikarenakan matriks piksel mengandung intensitas warna yang bisa menentukan konten ataupun pola yang ada dalam gambar.

## Modelling

Sekarang, mari kita coba beberapa model untuk mengklasifikasi apakah suatu gambar merupakan gambar kucing atau anjing. Data gambar yang akan digunakan adalah hasil pengambilan sampel dari data [Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats) dari Kaggle. Silakan download datanya melalui link [berikut](https://drive.google.com/file/d/1NC6u1-4eGXQPH5BFVXWLLJCCbp3Y4RD4/view?usp=sharing).

````{tip}
Setelah selesai mengunduh, sangat disarankan untuk diletakkan dalam folder `data` yang sejajar dengan notebook ini.
```
|- image-classification.ipynb
|- data
    |- dogs_vs_cats
        |- cat
            |- cat.1.jpg
            |- cat.2.jpg
            |- ...
        |- dog
            |- dog.1.jpg
            |- dog.2.jpg
            |- ...
```
````

```{note}
Kita akan coba latih beberapa model sekaligus, di antaranya:
* Logistic Regression
* SVM
* Decision Tree
* Random Forest
```


### Import Libraries

Pertama, kita impor terlebih dahulu beberapa library yang akan kita gunakan dalam notebook ini pada cell berikut.

```{note}
Untuk pemrosesan gambar, notebook ini akan menggunakan `OpenCV`. Jika kamu memilih menggunakan `Pillow` juga tidak masalah.
```

In [ ]:
import random
import time
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

### Data Loading

Untuk memuat semua gambar, baik kucing dan anjing, kita akan definisikan sebuah fungsi yang akan menerima lokasi gambar dalam bentuk *string*, kemudian mengembalikan gambar dalam format RGB.

```python
def load_rgb_image(image_path):
    """Load image from `image_path` to RGB image.
    
    Args:
        image_path (str): path to the image file.
            Must be in string.

    Returns:
        numpy.ndarray: image's pixel matrix in RGB.
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img
```

In [ ]:
def load_rgb_image(image_path):
    """Load image from `image_path` to RGB image.
    
    Args:
        image_path (str): path to the image file.
            Must be in string.

    Returns:
        numpy.ndarray: image's pixel matrix in RGB.
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
data_dir = Path("../../../../data/dogs_vs_cats")
cat_dir = data_dir / "cat"
dog_dir = data_dir / "dog"

dog_cat_filenames = (
    [file for file in cat_dir.glob("*.jpg")]
    + [file for file in dog_dir.glob("*.jpg")]
)

In [ ]:
dog_cat_images = {
    file.name: load_rgb_image(file.as_posix())
    for file in dog_cat_filenames
}

print(f"{len(dog_cat_images) = }")

Untuk menampilkan gambar kucing ataupun anjing, kita akan buat fungsi yang menerima `n` sebagai jumlah gambar yang akan ditampilkan secara acak.

```python
def show_n_images(n):
    """Show `n` images randomly.
    
    Args:
        n (int): number of images to be shown.
            Must be multiple of 2.
    """
    n_cols = n // 2
    sample_img = random.sample(list(dog_cat_images.items()), n)
    fig, ax = plt.subplots(2, n_cols, figsize=(n_cols*5, 6))
    for axis, img in zip(ax.flatten(), sample_img):
        axis.imshow(img[1])
        axis.set_title(f"{img[0].split('.')[0].title()} - {img[0]}")
        axis.set_yticks([])
        axis.set_xticks([])
    plt.show()
```

In [ ]:
def show_n_images(n):
    """Show `n` images randomly.
    
    Args:
        n (int): number of images to be shown.
            Must be multiple of 2.
    """
    n_cols = n // 2
    sample_img = random.sample(list(dog_cat_images.items()), n)
    fig, ax = plt.subplots(2, n_cols, figsize=(n_cols*5, 6))
    for axis, img in zip(ax.flatten(), sample_img):
        axis.imshow(img[1])
        axis.set_title(f"{img[0].split('.')[0].title()} - {img[0]}")
        axis.set_yticks([])
        axis.set_xticks([])
    plt.show()

In [ ]:
show_n_images(10)

Seperti yang kita lihat, ukuran gambar berbeda-beda. Karena kita harus membuat *feature image* dari data gambar tersebut, kita harus mengubah ukuran masing-masing gambar sedemikian hingga semua gambar memiliki ukurang yang sama.

In [ ]:
img_height = [img.shape[0] for img in dog_cat_images.values()]
img_width = [img.shape[1] for img in dog_cat_images.values()]

print(f"height stats: {np.max(img_height) = }.. "
      f"{np.min(img_height) = }.. {np.mean(img_height) = }.. "
      f"{np.median(img_height) = }")
print(f"width stats: {np.max(img_width) = }.. "
      f"{np.min(img_width) = }.. {np.mean(img_width) = }.. "
      f`"{np.median(img_width) = }")

Berdasarkan statistik di atas, kita akan mengubah semua gambar menjadi berukuran `64x64` menggunakan fungsi `cv2.resize`. Sehingga, jumlah total fitur gambar yang akan kita masukkan ke dalam model adalah 4096 fitur/piksel.

In [ ]:
image_dim = (64, 64)
for filename, image in dog_cat_images.items():
    dog_cat_images[filename] = cv2.resize(image, image_dim)

Jika kita hitung statistik gambar sekali lagi, maka baik *width* ataupun *height* akan berukuran sama.

In [ ]:


print(f"height stats: {np.max(img_height) = }.. {np.min(img_height) = }.. "
      f"{np.mean(img_height) = }.. {np.median(img_height) = }")
print(f"width stats: {np.max(img_width) = }.. {np.min(img_width) = }.. "
      f"{np.mean(img_width) = }.. {np.median(img_width) = }")

In [ ]:
show_n_images(10)

### Data Preparation

Sebelum kita masukkan data ke dalam model untuk *training*, kita perlu siapkan datanya terlebih dahulu. Ingat bahwa kita belum mempunyai target variabel, sehingga kita harus persiapkan terlebih dahulu. Kita bisa gunakan `dog_cat_images` untuk mendapatkan target variabel dari setiap nama file gambar menggunakan _list comprehension_ berikut.

```python
targets = [
    filename.split(".")[0]
    for filename in dog_cat_images.keys()
]
```

Kemudian, kita juga harus mendefinisikan data gambar yang berisi fitur-fitur gambar saja sebagai data yang akan kita gunakan untuk _training_. Sama seperti target variabel, kita bisa gunakan `dog_cat_images` dan _list comprehension_ berikut.

```python
features = np.array([
    cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    for img in dog_cat_images.values()
])
```

Untuk alasan penyederhanaan dan keterbatasan komputasi, kita juga akan ubah data gambar ke dalam format _grayscale_ sehingga hanya memiliki 1 channel warna saja.

In [ ]:
targets = [
    filename.split(".")[0]
    for filename in dog_cat_images.keys()
]

features = np.array([
    cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    for img in dog_cat_images.values()
])

In [ ]:
features.shape

Selanjutnya, kita akan ubah ukuran setiap gambar sedemikian hingga berdimensi 2. Lebih tepatnya, kita akan coba "ratakan" fitur gambar yang awalnya `(64, 64)` menjadi `4096` fitur yang sangat panjang.

In [ ]:
features = features.reshape((features.shape[0], -1))
print(f"New shape of features:", features.shape)

Kemudian, kita akan bagi data tersebut ke dalam data _trainig_ dan _development_.

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(
    features, targets,
    test_size=.2,
    random_state=11
)

print(f"{len(X_train) = }.. {len(X_dev) = }..")

### Logistic Regression

Model pertama yang akan kita coba adalah _Logistic Regression_.

In [ ]:
# define estimator
log_reg = LogisticRegression()
# log_reg = LogisticRegression(max_iter=500)
# log_reg = LogisticRegression(max_iter=1000)

# model training
print("Start training..")
start_time = time.time()
log_reg.fit(X_train, y_train)
print(f"Done training in {time.time() - start_time:.3f}s")

# prediction
log_reg_pred_train = log_reg.predict(X_train)
log_reg_pred_dev = log_reg.predict(X_dev)

# classification reports
print(
    "Performance on training set:",
    classification_report(y_train, log_reg_pred_train),
    sep="\n"
)
print(
    "Performance on development set:",
    classification_report(y_dev, log_reg_pred_dev),
    sep="\n"
)

### SVM

Model pertama yang akan kita coba adalah _Support Vector Machine_.

In [ ]:
# define estimator
svc = SVC()

# model training
print("Start training..")
start_time = time.time()
svc.fit(X_train, y_train)
print(f"Done training in {time.time() - start_time:.3f}s")

# prediction
svc_pred_train = svc.predict(X_train)
svc_pred_dev = svc.predict(X_dev)

# classification reports
print(
    "Performance on training set:",
    classification_report(y_train, svc_pred_train),
    sep="\n"
)
print(
    "Performance on development set:",
    classification_report(y_dev, svc_pred_dev),
    sep="\n"
)

### Decision Tree

Model pertama yang akan kita coba adalah _Decision Tree_.

In [ ]:
# define estimator
dtree = DecisionTreeClassifier()

# model training
print("Start training..")
start_time = time.time()
dtree.fit(X_train, y_train)
print(f"Done training in {time.time() - start_time:.3f}s")

# prediction
dtree_pred_train = dtree.predict(X_train)
dtree_pred_dev = dtree.predict(X_dev)

# classification reports
print(
    "Performance on training set:",
    classification_report(y_train, dtree_pred_train),
    sep="\n"
)
print(
    "Performance on development set:",
    classification_report(y_dev, dtree_pred_dev),
    sep="\n"
)

### Random Forest

Model pertama yang akan kita coba adalah _Decision Tree_.

In [ ]:
# define estimator
rf = RandomForestClassifier()

# model training
print("Start training..")
start_time = time.time()
rf.fit(X_train, y_train)
print(f"Done training in {time.time() - start_time:.3f}s")

# prediction
rf_pred_train = rf.predict(X_train)
rf_pred_dev = rf.predict(X_dev)

# classification reports
print(
    "Performance on training set:",
    classification_report(y_train, rf_pred_train),
    sep="\n"
)
print(
    "Performance on development set:",
    classification_report(y_dev, rf_pred_dev),
    sep="\n"
)

```{admonition} Eksplorasi
Kamu bisa eksplor lebih jauh lagi untuk mendapatkan dan menentukan model yang paling bagus, tidak _overfit_, apalagi _underfit_ menggunakan `GridSearch`.
```

## Fashion-MNIST

Setelah mencoba beberapa model pada dataset sebelumnya, sekarang giliran kamu untuk mengimplementasikannya menggunakan data Fashion-MNIST {cite:p}`xiao2017/online`.

<!-- ![](../../../../assets/images/fashion-mnist-sample.png) -->

```{figure} ../../../../assets/images/fashion-mnist-sample.png
:name: fashion-mnist-sample
:width: 50%

Sampel gambar dari data Fashion-MNIST ([ref](https://github.com/zalandoresearch/fashion-mnist)).
```

Untuk mengakses data, silakan unduh dataset melalui [Kaggle](https://www.kaggle.com/zalando-research/fashionmnist).

```{warning}
Harap perhatikan sisa kapasitas penyimpanan komputer kamu.
```

Terdapat $785$ kolom di mana kolom pertama adalah `label` yang merupakan target variabel. Sisanya adalah `pixel1` sampai `pixel784` yang menandakan bahwa setiap gambar pada data Fashion-MNIST berukuran `28x28` piksel atau total $784$ piksel.

Dari {cite:p}`xiao2017/online`, dataset Fashion-MNIST memiliki 10 kelas, yaitu:

(fashion-mnist-label)=

| Label | Description |
| :---: | :---------: |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

In [ ]:
# KETIK DI SINI

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2223ede1-8efb-486f-85ff-5fb31b0e703e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>